## COVID-19 : Predicting number of new cases in India and Kashmir

In [3]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM
from numpy import array
from numpy.random import uniform
from numpy import hstack
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.preprocessing import MinMaxScaler

Using TensorFlow backend.
d:\anaconda\envs\deeplearning\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
d:\anaconda\envs\deeplearning\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
d:\anaconda\envs\deeplearning\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
d:\anaconda\envs\deeplearning\lib\site-packages\tensorflow\python

### For India
The number of confirmed cases in India from 22 Jan 2020 to 28 May 2020 are utlized to train the model. The predictions are diaplyed for the dates ranging from 29 May 2020 to 05 June 2020.

#### Preprocessing the data

In [5]:
df = pd.read_csv("C:\\Users\\DELL\\Downloads\\494724_1196190_compressed_covid_19_data.csv\\covid_19_data.csv")


In [6]:
df.head(10)

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
0,1,01/22/2020,Anhui,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
1,2,01/22/2020,Beijing,Mainland China,1/22/2020 17:00,14.0,0.0,0.0
2,3,01/22/2020,Chongqing,Mainland China,1/22/2020 17:00,6.0,0.0,0.0
3,4,01/22/2020,Fujian,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
4,5,01/22/2020,Gansu,Mainland China,1/22/2020 17:00,0.0,0.0,0.0
5,6,01/22/2020,Guangdong,Mainland China,1/22/2020 17:00,26.0,0.0,0.0
6,7,01/22/2020,Guangxi,Mainland China,1/22/2020 17:00,2.0,0.0,0.0
7,8,01/22/2020,Guizhou,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
8,9,01/22/2020,Hainan,Mainland China,1/22/2020 17:00,4.0,0.0,0.0
9,10,01/22/2020,Hebei,Mainland China,1/22/2020 17:00,1.0,0.0,0.0


In [7]:
countries = ['India']

In [8]:
df_In = df[df['Country/Region'].isin(countries)]

In [9]:
len(df_In)

120

In [10]:
df_In.tail(10)

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
26410,26411,05/19/2020,NaN,India,2020-05-20 02:32:19,106475.0,3302.0,42309.0
26786,26787,05/20/2020,NaN,India,2020-05-21 02:32:48,112028.0,3434.0,45422.0
27238,27239,05/21/2020,NaN,India,2020-05-22 02:36:51,118226.0,3584.0,48553.0
27690,27691,05/22/2020,NaN,India,2020-05-23 02:32:42,124794.0,3726.0,51824.0
28142,28143,05/23/2020,NaN,India,2020-05-24 02:32:43,131423.0,3868.0,54385.0
28594,28595,05/24/2020,NaN,India,2020-05-25 02:32:44,138536.0,4024.0,57692.0
29046,29047,05/25/2020,NaN,India,2020-05-26 02:32:35,144950.0,4172.0,60706.0
29498,29499,05/26/2020,NaN,India,2020-05-27 02:32:31,150793.0,4344.0,64277.0
29950,29951,05/27/2020,NaN,India,2020-05-28 02:32:31,158086.0,4534.0,67749.0
30401,30402,05/28/2020,NaN,India,2020-05-29 02:32:50,165386.0,4711.0,70920.0


In [11]:
df_In = df_In.append(pd.DataFrame([{'Country/Region':'India','ObservationDate':'05/29/2020','Confirmed':173491.0}]))

In [12]:
df_In.tail()

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
29046,29047.0,05/25/2020,NaN,India,2020-05-26 02:32:35,144950.0,4172.0,60706.0
29498,29499.0,05/26/2020,NaN,India,2020-05-27 02:32:31,150793.0,4344.0,64277.0
29950,29951.0,05/27/2020,NaN,India,2020-05-28 02:32:31,158086.0,4534.0,67749.0
30401,30402.0,05/28/2020,NaN,India,2020-05-29 02:32:50,165386.0,4711.0,70920.0
0,NaN,05/29/2020,NaN,India,NaN,173491.0,NaN,NaN


In [13]:
df_In = df_In[['ObservationDate','Confirmed']]

In [282]:
test_data_size = 15

train_data = df_In[:-test_data_size]
test_data = df_In[-test_data_size:]

In [283]:
len(test_data)

15

In [284]:
train_data = train_data['Confirmed']
test_data = test_data['Confirmed']

In [286]:
import numpy as np

scaler = MinMaxScaler(feature_range=(0, 1))

scaler = scaler.fit(np.expand_dims(df_In['Confirmed'], axis=1))

train_data = scaler.transform(np.expand_dims(train_data, axis=1))

test_data = scaler.transform(np.expand_dims(test_data, axis=1))

In [287]:
test_data

array([[0.49445501],
       [0.52249121],
       [0.55159952],
       [0.57828693],
       [0.61371837],
       [0.64572598],
       [0.68145138],
       [0.71930947],
       [0.75751917],
       [0.79851865],
       [0.83548908],
       [0.86916825],
       [0.91120526],
       [0.95328261],
       [1.        ]])

In [288]:
def create_sequences(data, seq_length):
    xs = []
    ys = []

    for i in range(len(data)-seq_length-1):
        x = data[i:(i+seq_length)]
        y = data[i+seq_length]
        xs.append(x)
        ys.append(y)

    return np.array(xs), np.array(ys)

In [289]:
seq_length = 4
X_train, y_train = create_sequences(train_data, seq_length)
X_test, y_test = create_sequences(test_data, seq_length)

X_train = np.array(X_train).astype('float32')
y_train = np.array(y_train).astype('float32')

X_test = np.array(X_test).astype('float32')
y_test = np.array(y_test).astype('float32')

In [290]:
X_train.shape

(101, 4, 1)

In [291]:
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

In [296]:
train_generator = TimeseriesGenerator(train_data,train_data, length=2,sampling_rate=1, batch_size=10)     
test_generator = TimeseriesGenerator(test_data,test_data, length=2,sampling_rate=1, batch_size=1)

####  Training a Bidirectional LSTM model on the train data. Utlizing the validation and test sets.

In [293]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
from tensorflow.keras.layers import Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
from tensorflow.keras.optimizers import Adam
model = Sequential()
#model.add(Bidirectional(LSTM(512, input_shape=(5, 1),return_sequences=True)))
#model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(128,input_shape=(1, 1),return_sequences=True)))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(64,return_sequences=True)))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(1,activation='relu'))
model.compile(loss='mse', optimizer=Adam(),metric=['mae'])
# fit network
#model.fit_generator(train_generator, epochs=50, validation_data=(test_generator))

In [294]:
model.fit_generator(train_generator, epochs=100,steps_per_epoch=5,verbose=0)

In [297]:
model.evaluate_generator(test_generator)

0.0009336799535398873

In [298]:

y_test = model.predict_generator(test_generator)

In [299]:
y_test

array([[0.56675476],
       [0.5990227 ],
       [0.63045347],
       [0.6656241 ],
       [0.70354724],
       [0.7417186 ],
       [0.78309673],
       [0.82576734],
       [0.8701534 ],
       [0.91362697],
       [0.95293057],
       [0.9951508 ],
       [1.0417486 ]], dtype=float32)

In [300]:
test_data = scaler.inverse_transform(test_data)
train_data = scaler.inverse_transform(train_data)
prediction = scaler.inverse_transform(y_test)

In [301]:
import plotly.graph_objects as go
#prediction = model.predict_generator(test_generator)

close_train = train_data.reshape((-1))
close_test = test_data.reshape((-1))
prediction = prediction.reshape((-1))

trace1 = go.Scatter(
    x =  df_In['ObservationDate'][:-test_data_size],
    y = close_train,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = df_In['ObservationDate'][-test_data_size:],
    y = prediction,
    mode = 'lines',
    name = 'Prediction'
)
trace3 = go.Scatter(
    x = df_In['ObservationDate'][-test_data_size:],
    y = close_test,
    mode='lines',
    name = 'Ground Truth'
)
layout = go.Layout(
    title = "COVID-19 Spread",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Confirmed"}
)
fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()

In [578]:
model.save('model_lstm_1.h5')

### Making forecast of new cases in india at the end of a week's time (7 days)
Utlizing the entire data for training and then predicting for the number of new cases in future

In [173]:
#df = pd.read_csv("covid_19_clean_complete.csv")

train_data_ = df_In['Confirmed']

In [174]:
feature_range=(0, 1)

In [175]:
import numpy as np
scaler = MinMaxScaler(feature_range=(0, 1))
train_data_ = scaler.fit_transform(np.expand_dims(train_data_, axis=1))

#scaler = scaler.fit(np.expand_dims(train_data_, axis=1))

#train_data_ = scaler.transform(np.expand_dims(train_data_, axis=1))

In [176]:
train_data_.shape

(121, 1)

In [177]:
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
train_generator_ = TimeseriesGenerator(train_data_,train_data_, length=1, batch_size=3)

In [178]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
from tensorflow.keras.layers import Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
model_ = Sequential()
#model.add(Bidirectional(LSTM(512, input_shape=(5, 1),return_sequences=True)))
#model.add(Dropout(0.3))
model_.add(Bidirectional(LSTM(128,input_shape=(3, 1),activation='relu',return_sequences=True)))
model_.add(Dropout(0.3))
model_.add(Bidirectional(LSTM(64,activation='relu',return_sequences=True)))
model_.add(Dropout(0.3))
model_.add(Flatten())
model_.add(Dense(1,activation='relu'))
model_.compile(loss='mse', optimizer='adam')
model_.fit_generator(train_generator_, epochs=100,steps_per_epoch=5,verbose=0)

In [179]:
model_.evaluate_generator(train_generator_)

0.0005513135229762156

In [180]:
close_data_ = train_data_.reshape((-1))

In [181]:
model_.predict(close_data_[0].reshape(1,1,1)) 

array([[0.017854]], dtype=float32)

In [182]:
close_data_[-1].reshape((1,1,1)).shape

(1, 1, 1)

In [183]:
def predict(num_prediction, model):
    prediction_list = close_data_
    
    for _ in range(num_prediction):
        x = prediction_list[-1]
        x = x.reshape((1, 1, 1))
        out = model_.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    prediction_list = prediction_list[0:]
        
    return prediction_list
    
def predict_dates(num_prediction):
    last_date = df_In['ObservationDate'].values[-1]
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
    return prediction_dates

num_prediction = 7
forecast = predict(num_prediction, model_)
forecast_dates = predict_dates(num_prediction)

In [184]:
len(forecast_dates),len(forecast)

(8, 128)

In [185]:
forecast_dates

[Timestamp('2020-05-29 00:00:00', freq='D'),
 Timestamp('2020-05-30 00:00:00', freq='D'),
 Timestamp('2020-05-31 00:00:00', freq='D'),
 Timestamp('2020-06-01 00:00:00', freq='D'),
 Timestamp('2020-06-02 00:00:00', freq='D'),
 Timestamp('2020-06-03 00:00:00', freq='D'),
 Timestamp('2020-06-04 00:00:00', freq='D'),
 Timestamp('2020-06-05 00:00:00', freq='D')]

In [186]:
forecast[-8:]

array([1.        , 1.02782202, 1.06196845, 1.10443437, 1.15811431,
       1.22736704, 1.31905854, 1.44462752])

In [187]:
forecast = forecast[-8:].reshape(8,1)

In [188]:
len(forecast)

8

In [189]:
train_data_ = scaler.inverse_transform(train_data_)
forecast = scaler.inverse_transform(forecast)

In [190]:
forecast

array([[173491.        ],
       [178317.84184551],
       [184241.90565801],
       [191609.31902385],
       [200922.25234962],
       [212936.90837598],
       [228844.46566796],
       [250629.42903852]])

In [193]:
import plotly.graph_objects as go
#prediction = model.predict_generator(test_generator)

close_train = train_data_.reshape((-1))
forecast = forecast.reshape((-1))

trace1 = go.Scatter(
    x =  df_In['ObservationDate'],
    y = close_train,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = forecast_dates[1:],
    y = forecast[0:-1],
    mode = 'lines',
    name = 'Prediction'
)

layout = go.Layout(
    title = "COVID-19 Spread in India",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Confirmed"}
)
fig = go.Figure(data=[trace1, trace2], layout=layout)
fig.show()

### Covid 19 Spread in J&K

#### Preprocessing data

In [194]:
df = pd.read_csv("C:\\Users\\DELL\\Downloads\\datasets_557629_1198871_covid_19_india.csv")

In [195]:
df.head()

,Sno,Date,Time,State/UnionTerritory,ConfirmedIndianNational,ConfirmedForeignNational,Cured,Deaths,Confirmed
0,1,30/01/20,6:00 PM,Kerala,1,0,0,0,1
1,2,31/01/20,6:00 PM,Kerala,1,0,0,0,1
2,3,01/02/20,6:00 PM,Kerala,2,0,0,0,2
3,4,02/02/20,6:00 PM,Kerala,3,0,0,0,3
4,5,03/02/20,6:00 PM,Kerala,3,0,0,0,3


In [196]:
df['State/UnionTerritory'].unique()

array(['Kerala', 'Telengana', 'Delhi', 'Rajasthan', 'Uttar Pradesh',
       'Haryana', 'Ladakh', 'Tamil Nadu', 'Karnataka', 'Maharashtra',
       'Punjab', 'Jammu and Kashmir', 'Andhra Pradesh', 'Uttarakhand',
       'Odisha', 'Puducherry', 'West Bengal', 'Chhattisgarh',
       'Chandigarh', 'Gujarat', 'Himachal Pradesh', 'Madhya Pradesh',
       'Bihar', 'Manipur', 'Mizoram', 'Andaman and Nicobar Islands',
       'Goa', 'Unassigned', 'Assam', 'Jharkhand', 'Arunachal Pradesh',
       'Tripura', 'Nagaland', 'Meghalaya', 'Dadar Nagar Haveli',
       'Cases being reassigned to states', 'Sikkim'], dtype=object)

In [198]:
state = ['Jammu and Kashmir']

In [199]:
df_jk = df[df['State/UnionTerritory'].isin(state)]

In [200]:
df_jk = df_jk[['Date','Confirmed']]

In [201]:
df_jk.head()

,Date,Confirmed
81,09/03/20,1
96,10/03/20,1
106,11/03/20,1
117,12/03/20,1
130,13/03/20,1


In [205]:
len(df_jk)

83

In [208]:
test_data_size = 14

train_data = df_jk['Confirmed'][:-test_data_size]
test_data = df_jk['Confirmed'][-test_data_size:]

In [ ]:
import numpy as np

scaler = MinMaxScaler(feature_range=(0, 1))

scaler = scaler.fit(np.expand_dims(df_jk['Confirmed'], axis=1))

train_data = scaler.transform(np.expand_dims(train_data, axis=1))

test_data = scaler.transform(np.expand_dims(test_data, axis=1))

In [ ]:
train_generator = TimeseriesGenerator(train_data,train_data, length=2,sampling_rate=1, batch_size=3)     
test_generator = TimeseriesGenerator(test_data,test_data, length=2,sampling_rate=1, batch_size=1)

### Training a Bidirectional LSTM model

In [211]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
from tensorflow.keras.layers import Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
from tensorflow.keras.optimizers import Adam
model = Sequential()
#model.add(Bidirectional(LSTM(512, input_shape=(5, 1),return_sequences=True)))
#model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(128,input_shape=(1, 1),return_sequences=True)))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(64,return_sequences=True)))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(1,activation='relu'))
model.compile(loss='mse', optimizer=Adam(),metric=['mae'])

In [212]:
model.fit_generator(train_generator, epochs=80,steps_per_epoch=5,verbose=0)

In [213]:
model.evaluate_generator(test_generator)

0.0010579510418817033

In [214]:
y_test = model.predict_generator(test_generator)

In [215]:
test_data = scaler.inverse_transform(test_data)
train_data = scaler.inverse_transform(train_data)
prediction = scaler.inverse_transform(y_test)

In [217]:
import plotly.graph_objects as go
#prediction = model.predict_generator(test_generator)

close_train = train_data.reshape((-1))
close_test = test_data.reshape((-1))
prediction = prediction.reshape((-1))

trace1 = go.Scatter(
    x =  df_jk['Date'][:-test_data_size],
    y = close_train,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = df_jk['Date'][-test_data_size:],
    y = prediction,
    mode = 'lines',
    name = 'Prediction'
)
trace3 = go.Scatter(
    x = df_jk['Date'][-test_data_size:],
    y = close_test,
    mode='lines',
    name = 'Ground Truth'
)
layout = go.Layout(
    title = "COVID-19 Spread",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Confirmed"}
)
fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()

### Making forecast of new cases in Kashmir at the end of a week's time (7 days)
Utlizing the entire data for training and then predicting for the number of new cases in future

In [508]:
train_data_ = df_jk['Confirmed']

In [509]:
len(train_data_)

83

In [510]:
import numpy as np
scaler = MinMaxScaler(feature_range=(0, 1))
train_data_ = scaler.fit_transform(np.expand_dims(train_data_, axis=1))

In [511]:
train_data_.shape

(83, 1)

In [512]:
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
train_generator_ = TimeseriesGenerator(train_data_,train_data_, length=1, batch_size=5)

In [513]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
from tensorflow.keras.layers import Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
model_ = Sequential()
#model.add(Bidirectional(LSTM(512, input_shape=(5, 1),return_sequences=True)))
model.add(Dropout(0.3))
model_.add(Bidirectional(LSTM(32,input_shape=(3, 1),activation='relu',return_sequences=True)))
model_.add(Dropout(0.3))
model_.add(Bidirectional(LSTM(64,activation='relu',return_sequences=True)))
#model_.add(Dropout(0.3))
model_.add(Flatten())
model_.add(Dense(1,activation='relu'))
model_.compile(loss='mse', optimizer='adam')
model_.fit_generator(train_generator_, epochs=100,steps_per_epoch=5,verbose=0)

In [514]:
model_.evaluate_generator(train_generator_)

0.001724789266304202

In [515]:
close_data_ = train_data_.reshape((-1))

In [516]:
def predict(num_prediction, model):
    prediction_list = close_data_
    
    for _ in range(num_prediction):
        x = prediction_list[-1]
        x = x.reshape((1, 1, 1))
        out = model_.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    prediction_list = prediction_list[0:]
        
    return prediction_list
    
def predict_dates(num_prediction):
    last_date = df_jk['Date'].values[-1]
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
    return prediction_dates

num_prediction = 7
forecast = predict(num_prediction, model_)
forecast_dates = predict_dates(num_prediction)

In [517]:
len(forecast_dates),len(forecast)

(8, 90)

In [518]:
forecast_dates

[Timestamp('2020-05-30 00:00:00', freq='D'),
 Timestamp('2020-05-31 00:00:00', freq='D'),
 Timestamp('2020-06-01 00:00:00', freq='D'),
 Timestamp('2020-06-02 00:00:00', freq='D'),
 Timestamp('2020-06-03 00:00:00', freq='D'),
 Timestamp('2020-06-04 00:00:00', freq='D'),
 Timestamp('2020-06-05 00:00:00', freq='D'),
 Timestamp('2020-06-06 00:00:00', freq='D')]

In [519]:
forecast = forecast[-8:].reshape(8,1)

In [520]:
train_data_ = scaler.inverse_transform(train_data_)
forecast = scaler.inverse_transform(forecast)

In [521]:
forecast


array([[2164.        ],
       [2258.43770802],
       [2387.71168315],
       [2570.3605448 ],
       [2840.0205276 ],
       [3264.22581828],
       [3999.30965447],
       [5494.15973091]])

In [522]:
import plotly.graph_objects as go
#prediction = model.predict_generator(test_generator)

close_train = train_data_.reshape((-1))
forecast = forecast.reshape((-1))

trace1 = go.Scatter(
    x =  df_jk['Date'],
    y = close_train,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = forecast_dates[1:],
    y = forecast[0:-1],
    mode = 'lines',
    name = 'Prediction'
)

layout = go.Layout(
    title = "COVID-19 Spread in J&K",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Confirmed"}
)
fig = go.Figure(data=[trace1, trace2], layout=layout)
fig.show()